In [ ]:
import torch
import pickle
import gzip
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import torch.utils.data as data
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import random
import sklearn.utils.class_weight

In [ ]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Device: cuda:0


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dt = shuffle(pd.read_pickle('/content/drive/MyDrive/train.pkl'))

In [ ]:
x = []
y = []
for element in dt:
  x.append(torch.tensor(element[0]).float())
  y.append(element[1])


In [ ]:
x_padded = pad_sequence(x, batch_first=True)
len(x_padded[0])

6308

In [ ]:
train_indices = int(0.7*len(y))

train_set = data.TensorDataset(torch.tensor(x_padded[:train_indices]), torch.tensor(y[:train_indices]))
train_loader = data.DataLoader(train_set, batch_size=32)
test_data, test_targets = x_padded[train_indices:], y[train_indices:]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
class LSTMRegressor(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, out_size, bidirectional = False):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional = bidirectional, dropout=0.4)
        if bidirectional:
          self.bidirectional = 2
        else:
          self.bidirectional = 1
        self.fc = nn.Linear(hidden_size*len(x_padded[0])*self.bidirectional, out_size)

        
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers*self.bidirectional, batch_size, self.hidden_size)
        state = torch.zeros(self.num_layers*self.bidirectional, batch_size, self.hidden_size)
        return hidden, state
    
    def forward(self, x, hidden):
        x = torch.transpose(x,0,1)
        all_outputs, hidden = self.lstm(x, hidden)
        all_outputs = torch.transpose(all_outputs,0,1)
        out = torch.flatten(all_outputs, 1)
        x = self.fc(out)
        return x, hidden


model = LSTMRegressor(1,20,3,5,False).to(device)

In [ ]:
class_weights = sklearn.utils.class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(y),y=y)
class_weights = torch.tensor(class_weights).float()

In [ ]:
def count_accuracy():
  with torch.no_grad():
    hidden, state = model.init_hidden(len(test_data))
    hidden, state = hidden.to(device), state.to(device) 
    preds, _ = model(test_data.to(device).unsqueeze(2),(hidden, state))
  p = torch.argmax(preds,1).cpu()
  counter = 0
  for i in range(len(test_targets)):
    if p[i] == test_targets[i]:
      counter += 1
  return counter/len(test_targets)


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
loss_fun = nn.CrossEntropyLoss(weight = class_weights.to(device))
last_acc = 0
# Training loop
for epoch in range(30):
    for x, targets in train_loader:
        x = x.to(device).unsqueeze(2)
        targets = targets.to(device)
        hidden, state = model.init_hidden(x.size(0))
        hidden, state = hidden.to(device), state.to(device) 
        preds, _ = model(x, (hidden,state))
        preds = preds.squeeze(1)
        optimizer.zero_grad() 
        loss = loss_fun(preds, targets)
        loss.backward()
        optimizer.step()
    acc = count_accuracy()
    print(f"Epoch: {epoch}, loss: {loss.item():.3} acc: {acc:.3}")
    if acc - last_acc < -0.1:
      break
    last_acc = acc

KeyboardInterrupt: ignored

In [ ]:
# model = LSTMRegressor(1,10,2,5,True).to(device)

In [ ]:
with torch.no_grad():
    hidden, state = model.init_hidden(len(test_data))
    hidden, state = hidden.to(device), state.to(device) 
    preds, _ = model(test_data.to(device).unsqueeze(2),(hidden, state))
# print(f"Accuracy: {(torch.argmax(preds,1).cpu() == test_targets).sum().item()/len(test_targets)}")

In [ ]:
p = torch.argmax(preds,1).cpu()
counter = 0
for i in range(len(test_targets)):
  if p[i] == test_targets[i]:
    counter += 1
print(counter/len(test_targets))


0.4886621315192744


In [ ]:
def save_results(tensor):
  predictions = tensor.cpu().detach().numpy()
  pd.DataFrame(predictions).to_csv("MatakGromadzka.csv",header=False, index=False)